From: https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

In [ ]:
tf.__version__

In [ ]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

In [ ]:
dataframe.head()

In [ ]:
# In the original dataset, `'AdoptionSpeed'` of `4` indicates
# a pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop unused features.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

In [ ]:
train, val, test = np.split(dataframe.sample(frac=1), [int(0.8*len(dataframe)), int(0.9*len(dataframe))])


In [ ]:
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('target')
    df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [ ]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

In [ ]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of ages:', train_features['Age'])
print('A batch of targets:', label_batch )

In [ ]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

In [ ]:
photo_count_col = train_features['PhotoAmt']
layer = get_normalization_layer('PhotoAmt', train_ds)
layer(photo_count_col)

In [ ]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # Create a layer that turns strings into integer indices.
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    # Otherwise, create a layer that turns integer values into integer indices.
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)

    # Prepare a `tf.data.Dataset` that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)

    # Encode the integer indices.
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    return lambda feature: encoder(index(feature))

In [ ]:
test_type_col = train_features['Type']
test_type_layer = get_category_encoding_layer(name='Type',
                                              dataset=train_ds,
                                              dtype='string')
test_type_layer(test_type_col)

In [ ]:
test_age_col = train_features['Age']
test_age_layer = get_category_encoding_layer(name='Age',
                                             dataset=train_ds,
                                             dtype='int64',
                                             max_tokens=5)
test_age_layer(test_age_col)

In [ ]:
batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
all_inputs = []
encoded_features = []

# Numerical features.
for header in ['PhotoAmt', 'Fee']:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

In [ ]:
age_col = tf.keras.Input(shape=(1,), name='Age', dtype='int64')

encoding_layer = get_category_encoding_layer(name='Age',
                                             dataset=train_ds,
                                             dtype='int64',
                                             max_tokens=5)
encoded_age_col = encoding_layer(age_col)
all_inputs.append(age_col)
encoded_features.append(encoded_age_col)

In [ ]:
categorical_cols = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                    'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Breed1']

for header in categorical_cols:
    categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
    encoding_layer = get_category_encoding_layer(name=header,
                                                 dataset=train_ds,
                                                 dtype='string',
                                                 max_tokens=5)
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

In [ ]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
# Use `rankdir='LR'` to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

In [ ]:
model.save('my_pet_classifier')

In [ ]:
reloaded_model = tf.keras.models.load_model('my_pet_classifier')

In [ ]:
reloaded_model.inputs

In [ ]:
sample = {
    'Type': 'Cat',
    'Age': 3,
    'Breed1': 'Tabby',
    'Gender': 'Male',
    'Color1': 'Black',
    'Color2': 'White',
    'MaturitySize': 'Small',
    'FurLength': 'Short',
    'Vaccinated': 'No',
    'Sterilized': 'No',
    'Health': 'Healthy',
    'Fee': 100,
    'PhotoAmt': 2,
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = reloaded_model.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print(
    "This particular pet had a %.1f percent probability "
    "of getting adopted." % (100 * prob)
)

## PySpark

In [ ]:
df = spark.createDataFrame(dataframe)

In [ ]:
df.write.mode("overwrite").parquet("datasets/petfinder-mini")

In [ ]:
df.show()

## Inference using Spark ML Model

In [ ]:
import sparkext

In [ ]:
df = spark.read.parquet("datasets/petfinder-mini")

In [ ]:
df.show()

In [ ]:
columns = df.columns
print(columns)

In [ ]:
# remove label column
columns.remove("target")
print(columns)

In [ ]:
my_model = sparkext.tensorflow.Model("my_pet_classifier") \
                .setInputCols(columns) \
                .setOutputCol("pred")

In [ ]:
%%time
predictions = my_model.transform(df)
results = predictions.collect()

In [ ]:
predictions.show()

## Inference using Spark DL UDF

### Spark DataFrame column names match model input names

In [ ]:
df = spark.read.parquet("datasets/petfinder-mini")

In [ ]:
df.show()

In [ ]:
columns = df.columns
print(columns)

In [ ]:
# remove label column
columns.remove("target")
print(columns)

In [ ]:
from sparkext.tensorflow import model_udf

In [ ]:
# need to pass the list of columns into the model_udf
classify = model_udf("my_pet_classifier", input_columns=columns)

In [ ]:
df.withColumn("preds", classify(*columns)).show(truncate=10)

In [ ]:
%%time
results = df.withColumn("preds", classify(*columns)).collect()

### Simulate Spark DataFrame column names not matching model input names

In [ ]:
df2 = df.withColumnRenamed("Type", "species") \
        .withColumnRenamed("Age", "years") \
        .withColumnRenamed("Breed1", "breed") \
        .withColumnRenamed("Gender", "sex") \
        .withColumnRenamed("Color1", "main_color") \
        .withColumnRenamed("Color2", "secondary_color") \
        .withColumnRenamed("MaturitySize", "full_size") \
        .withColumnRenamed("FurLength", "fur_length") \
        .withColumnRenamed("Vaccinated", "immunized") \
        .withColumnRenamed("Sterilized", "spayed") \
        .withColumnRenamed("Health", "health") \
        .withColumnRenamed("Fee", "cost") \
        .withColumnRenamed("PhotoAmt","photos") \
        .withColumnRenamed("target", "adopted")
df2.show(truncate=10)

In [ ]:
spark_columns = df2.columns
spark_columns.remove('adopted')
spark_columns

In [ ]:
# User must provide ordered list of equivalent model input names 
model_columns = ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']
model_columns

In [ ]:
# pass the list of model input names into the `model_udf` helper
classify = model_udf("my_pet_classifier", input_columns=model_columns)

In [ ]:
# pass the list of Spark columns as the Spark SQL selectors
df2.withColumn("preds", classify(*spark_columns)).show(truncate=10)

## Inference using Spark DL API

In [1]:
import numpy as np

from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct, col
from pyspark.sql.types import ArrayType, FloatType
from typing import Union, Dict

In [2]:
df = spark.read.parquet("datasets/petfinder-mini")

In [3]:
df.show(5)

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized| Health|Fee|PhotoAmt|target|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
| Cat|  3|               Tabby|  Male| Black|   White|       Small|    Short|        No|        No|Healthy|100|       1|     1|
| Cat|  1|Domestic Medium Hair|  Male| Black|   Brown|      Medium|   Medium|  Not Sure|  Not Sure|Healthy|  0|       2|     1|
| Dog|  1|         Mixed Breed|  Male| Brown|   White|      Medium|   Medium|       Yes|        No|Healthy|  0|       7|     1|
| Dog|  4|         Mixed Breed|Female| Black|   Brown|      Medium|    Short|       Yes|        No|Healthy|150|       8|     1|
| Dog|  1|         Mixed Breed|  Male| Black|No Color|      Medium|    Short|        No|        No|Healt

In [4]:
columns = df.columns
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']


In [5]:
# remove label column
columns.remove("target")
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']


In [6]:
def predict_batch_fn():
    import tensorflow as tf
    model = tf.keras.models.load_model('/home/leey/devpub/leewyang/sparkext/examples/tensorflow/my_pet_classifier')

    def predict(t, a, b, g, c1, c2, m, f, v, s, h, fee, p):
        inputs = {
            "Type": t,
            "Age": a,
            "Breed1": b,
            "Gender": g,
            "Color1": c1,
            "Color2": c2,
            "MaturitySize": m,
            "FurLength": f,
            "Vaccinated": v,
            "Sterilized": s,
            "Health": h,
            "Fee": fee,
            "PhotoAmt": p
        }
        return model.predict(inputs)

    return predict

In [7]:
# need to pass the list of columns into the model_udf
classify = predict_batch_udf(predict_batch_fn,
                             return_type=FloatType(),
                             batch_size=100)

In [8]:
%%time
results = df.withColumn("preds", classify(struct(*columns)))
results.collect()

22/09/21 12:44:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CPU times: user 148 ms, sys: 0 ns, total: 148 ms
Wall time: 12.4 s


[Row(Type='Cat', Age=3, Breed1='Tabby', Gender='Male', Color1='Black', Color2='White', MaturitySize='Small', FurLength='Short', Vaccinated='No', Sterilized='No', Health='Healthy', Fee=100, PhotoAmt=1, target=1, preds=1.4765713214874268),
 Row(Type='Cat', Age=1, Breed1='Domestic Medium Hair', Gender='Male', Color1='Black', Color2='Brown', MaturitySize='Medium', FurLength='Medium', Vaccinated='Not Sure', Sterilized='Not Sure', Health='Healthy', Fee=0, PhotoAmt=2, target=1, preds=1.2110843658447266),
 Row(Type='Dog', Age=1, Breed1='Mixed Breed', Gender='Male', Color1='Brown', Color2='White', MaturitySize='Medium', FurLength='Medium', Vaccinated='Yes', Sterilized='No', Health='Healthy', Fee=0, PhotoAmt=7, target=1, preds=2.21209454536438),
 Row(Type='Dog', Age=4, Breed1='Mixed Breed', Gender='Female', Color1='Black', Color2='Brown', MaturitySize='Medium', FurLength='Short', Vaccinated='Yes', Sterilized='No', Health='Healthy', Fee=150, PhotoAmt=8, target=1, preds=0.5492994785308838),
 Row(T

In [9]:
%%time
results = df.withColumn("preds", classify(*columns))
results.collect()

CPU times: user 60.3 ms, sys: 12.7 ms, total: 73 ms
Wall time: 6.45 s


[Row(Type='Cat', Age=3, Breed1='Tabby', Gender='Male', Color1='Black', Color2='White', MaturitySize='Small', FurLength='Short', Vaccinated='No', Sterilized='No', Health='Healthy', Fee=100, PhotoAmt=1, target=1, preds=1.4765713214874268),
 Row(Type='Cat', Age=1, Breed1='Domestic Medium Hair', Gender='Male', Color1='Black', Color2='Brown', MaturitySize='Medium', FurLength='Medium', Vaccinated='Not Sure', Sterilized='Not Sure', Health='Healthy', Fee=0, PhotoAmt=2, target=1, preds=1.2110843658447266),
 Row(Type='Dog', Age=1, Breed1='Mixed Breed', Gender='Male', Color1='Brown', Color2='White', MaturitySize='Medium', FurLength='Medium', Vaccinated='Yes', Sterilized='No', Health='Healthy', Fee=0, PhotoAmt=7, target=1, preds=2.21209454536438),
 Row(Type='Dog', Age=4, Breed1='Mixed Breed', Gender='Female', Color1='Black', Color2='Brown', MaturitySize='Medium', FurLength='Short', Vaccinated='Yes', Sterilized='No', Health='Healthy', Fee=150, PhotoAmt=8, target=1, preds=0.5492994785308838),
 Row(T

In [10]:
%%time
results = df.withColumn("preds", classify(*[col(c) for c in columns]))
results.collect()

CPU times: user 51.8 ms, sys: 13.9 ms, total: 65.7 ms
Wall time: 6.57 s


[Row(Type='Cat', Age=3, Breed1='Tabby', Gender='Male', Color1='Black', Color2='White', MaturitySize='Small', FurLength='Short', Vaccinated='No', Sterilized='No', Health='Healthy', Fee=100, PhotoAmt=1, target=1, preds=1.4765713214874268),
 Row(Type='Cat', Age=1, Breed1='Domestic Medium Hair', Gender='Male', Color1='Black', Color2='Brown', MaturitySize='Medium', FurLength='Medium', Vaccinated='Not Sure', Sterilized='Not Sure', Health='Healthy', Fee=0, PhotoAmt=2, target=1, preds=1.2110843658447266),
 Row(Type='Dog', Age=1, Breed1='Mixed Breed', Gender='Male', Color1='Brown', Color2='White', MaturitySize='Medium', FurLength='Medium', Vaccinated='Yes', Sterilized='No', Health='Healthy', Fee=0, PhotoAmt=7, target=1, preds=2.21209454536438),
 Row(Type='Dog', Age=4, Breed1='Mixed Breed', Gender='Female', Color1='Black', Color2='Brown', MaturitySize='Medium', FurLength='Short', Vaccinated='Yes', Sterilized='No', Health='Healthy', Fee=150, PhotoAmt=8, target=1, preds=0.5492994785308838),
 Row(T

In [11]:
results.show()

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+------------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized| Health|Fee|PhotoAmt|target|       preds|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+------------+
| Cat|  3|               Tabby|  Male| Black|   White|       Small|    Short|        No|        No|Healthy|100|       1|     1|   1.4765713|
| Cat|  1|Domestic Medium Hair|  Male| Black|   Brown|      Medium|   Medium|  Not Sure|  Not Sure|Healthy|  0|       2|     1|   1.2110844|
| Dog|  1|         Mixed Breed|  Male| Brown|   White|      Medium|   Medium|       Yes|        No|Healthy|  0|       7|     1|   2.2120945|
| Dog|  4|         Mixed Breed|Female| Black|   Brown|      Medium|    Short|       Yes|        No|Healthy|150|       8|     1|   0.5492995|
| Dog|  1|   

### Using Triton Server

In [12]:
import numpy as np

from functools import partial
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct
from pyspark.sql.types import ArrayType, FloatType
from typing import Union, Dict

In [13]:
df = spark.read.parquet("datasets/petfinder-mini")

In [14]:
df.show(5)

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized| Health|Fee|PhotoAmt|target|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
| Cat|  3|               Tabby|  Male| Black|   White|       Small|    Short|        No|        No|Healthy|100|       1|     1|
| Cat|  1|Domestic Medium Hair|  Male| Black|   Brown|      Medium|   Medium|  Not Sure|  Not Sure|Healthy|  0|       2|     1|
| Dog|  1|         Mixed Breed|  Male| Brown|   White|      Medium|   Medium|       Yes|        No|Healthy|  0|       7|     1|
| Dog|  4|         Mixed Breed|Female| Black|   Brown|      Medium|    Short|       Yes|        No|Healthy|150|       8|     1|
| Dog|  1|         Mixed Breed|  Male| Black|No Color|      Medium|    Short|        No|        No|Healt

In [15]:
columns = df.columns
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']


In [16]:
# remove label column
columns.remove("target")
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']


#### Start Triton Server on each executor

In [17]:
num_executors = 1

nodeRDD = sc.parallelize(list(range(num_executors)), num_executors)

def start_triton(it):
    import docker
    import time
    import tritonclient.grpc as grpcclient
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    if containers:
        print(">>>> containers: {}".format([c.short_id for c in containers]))
    else:
        container=client.containers.run(
            "nvcr.io/nvidia/tritonserver:22.07-py3", "tritonserver --model-repository=/models",
            detach=True,
            device_requests=[docker.types.DeviceRequest(device_ids=["0"], capabilities=[['gpu']])],
            name="spark-triton",
            network_mode="host",
            remove=True,
            shm_size="64M",
            volumes={"/home/leey/devpub/leewyang/sparkext/examples/models": {"bind": "/models", "mode": "ro"}}
        )
        print(">>>> starting triton: {}".format(container.short_id))

        # wait for triton to be running
        time.sleep(15)
        client = grpcclient.InferenceServerClient("localhost:8001")
        ready = False
        while not ready:
            try:
                ready = client.is_server_ready()
            except Exception as e:
                time.sleep(5)
            
    return [True]

nodeRDD.mapPartitions(start_triton).collect()

[True]

#### Run inference

In [18]:
def triton_fn(triton_uri, model_name):
    import numpy as np
    import tritonclient.grpc as grpcclient
    
    np_types = {
      "BOOL": np.dtype(np.bool8),
      "INT8": np.dtype(np.int8),
      "INT16": np.dtype(np.int16),
      "INT32": np.dtype(np.int32),
      "INT64": np.dtype(np.int64),
      "FP16": np.dtype(np.float16),
      "FP32": np.dtype(np.float32),
      "FP64": np.dtype(np.float64),
      "FP64": np.dtype(np.double),
      "BYTES": np.dtype(object)
    }

    client = grpcclient.InferenceServerClient(triton_uri)
    model_meta = client.get_model_metadata(model_name)
    
    def predict(t, a, b, g, c1, c2, m, f, v, s, h, fee, p):
        # convert input ndarrays into a dictionary of ndarrays
        inputs = {
            "Type": t, 
            "Age": a, 
            "Breed1": b, 
            "Gender": g,
            "Color1": c1,
            "Color2": c2,
            "MaturitySize": m,
            "FurLength": f,
            "Vaccinated": v, 
            "Sterilized": s,
            "Health": h,
            "Fee": fee,
            "PhotoAmt": p
        }
        return _predict(inputs)
        
    def _predict(inputs):
        if isinstance(inputs, np.ndarray):
            # single ndarray input
            request = [grpcclient.InferInput(model_meta.inputs[0].name, inputs.shape, model_meta.inputs[0].datatype)]
            request[0].set_data_from_numpy(inputs.astype(np_types[model_meta.inputs[0].datatype]))
        else:
            # dict of multiple ndarray inputs
            request = [grpcclient.InferInput(i.name, inputs[i.name].shape, i.datatype) for i in model_meta.inputs]
            for i in request:
                i.set_data_from_numpy(inputs[i.name()].astype(np_types[i.datatype()]))
        
        response = client.infer(model_name, inputs=request)
        
        if len(model_meta.outputs) > 1:
            # return dictionary of numpy arrays
            return {o.name: response.as_numpy(o.name) for o in model_meta.outputs}
        else:
            # return single numpy array
            return response.as_numpy(model_meta.outputs[0].name)
        
        
    return predict

In [19]:
# need to pass the list of columns into the model_udf
classify = predict_batch_udf(partial(triton_fn, triton_uri="localhost:8001", model_name="my_pet_classifier"),
                             input_tensor_shapes=[[1]] * len(columns),
                             return_type=FloatType(),
                             batch_size=100)

In [20]:
# FAILS: Op type not registered 'DenseBincount'
df.withColumn("preds", classify(struct(*columns))).show(truncate=10)

22/09/21 12:45:25 WARN TaskSetManager: Lost task 0.0 in stage 9.0 (TID 12) (192.168.86.223 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/leey/devpub/leewyang/spark/python/pyspark/ml/functions.py", line 292, in predict
    preds = predict_fn(*multi_inputs)
  File "/tmp/ipykernel_223318/3574973703.py", line 38, in predict
  File "/tmp/ipykernel_223318/3574973703.py", line 51, in _predict
  File "/home/leey/.pyenv/versions/spark_dev/lib/python3.9/site-packages/tritonclient/grpc/__init__.py", line 1322, in infer
    raise_error_grpc(rpc_error)
  File "/home/leey/.pyenv/versions/spark_dev/lib/python3.9/site-packages/tritonclient/grpc/__init__.py", line 62, in raise_error_grpc
    raise get_error_grpc(rpc_error) from None
tritonclient.utils.InferenceServerException: [StatusCode.INTERNAL] [_Derived_]{{function_node __inference_signature_wrapper_21132}} {{function_node __inference_signature_wrapper_21132}} Op type not registered 'Den

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/leey/devpub/leewyang/spark/python/pyspark/ml/functions.py", line 292, in predict
    preds = predict_fn(*multi_inputs)
  File "/tmp/ipykernel_223318/3574973703.py", line 38, in predict
  File "/tmp/ipykernel_223318/3574973703.py", line 51, in _predict
  File "/home/leey/.pyenv/versions/spark_dev/lib/python3.9/site-packages/tritonclient/grpc/__init__.py", line 1322, in infer
    raise_error_grpc(rpc_error)
  File "/home/leey/.pyenv/versions/spark_dev/lib/python3.9/site-packages/tritonclient/grpc/__init__.py", line 62, in raise_error_grpc
    raise get_error_grpc(rpc_error) from None
tritonclient.utils.InferenceServerException: [StatusCode.INTERNAL] [_Derived_]{{function_node __inference_signature_wrapper_21132}} {{function_node __inference_signature_wrapper_21132}} Op type not registered 'DenseBincount' in binary running on leey-dt. Make 

#### Stop Triton Server on each executor

In [21]:
def stop_triton(it):
    import docker
    import time
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    print(">>>> stopping containers: {}".format([c.short_id for c in containers]))
    if containers:
        container=containers[0]
        container.stop(timeout=120)

    return [True]

nodeRDD.mapPartitions(stop_triton).collect()

[True]

In [22]:
spark.stop()